# Files and versioning

Unless you are a string theorist, at some point you're probably going to want to save and load some data. This tutorial covers some of Sciris' tools for doing that more easily.

<div class="alert alert-info">
    
Click [here](https://mybinder.org/v2/gh/sciris/sciris/HEAD?labpath=docs%2Ftutorials%2Ftut_files.ipynb) to open an interactive version of this notebook.

</div>


<div class="alert alert-warning">
    <b>Warning!</b> The tools here are powerful, which also makes them dangerous. Unless it's in a simple text format like JSON or CSV, loading a datafile can run arbitrary code on your computer, just like running a Python script can. If you wouldn't run a Python file from some source, don't open a data file from that source either.
</div>

## Saving and loading arbitrary objects

Let's assume you're mostly just saving and loading files you've created yourself or from trusted colleagues, not opening email attachments from the branch of the local mafia. Then everything here is absolutely fine.

Let's revisit our sim from the first tutorial:

In [ ]:
import sciris as sc
import numpy as np
import pylab as pl
sc.options(jupyter=True) # To make plots nicer

class Sim:
    
    def __init__(self, days, trials):
        self.days = days
        self.trials = trials
    
    def run(self):
        self.x = np.arange(self.days)
        self.y = np.cumsum(np.random.randn(self.days, self.trials)**3, axis=0)
    
    def plot(self):
        with pl.style.context('sciris.fancy'):
            pl.plot(self.x, self.y, alpha=0.6)

# Run and save
sim = Sim(days=100, trials=10)
sim.run()
sc.save('my-sim.obj', sim) # Save any Python object to disk

# Load and plot
new_sim = sc.load('my-sim.obj') # Load any Python object
new_sim.plot()

In [ ]:
print(sc.jsonify(sim, indent=2, tostring=True))